In [1]:
!pip install regex
import pandas as pd
import os
import numpy as np
import re

You should consider upgrading via the 'C:\Users\juanm\OneDrive\Escritorio\Python\SQL---Coderhouse---Proyecto-Final\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
os.chdir('C:\\Users\\juanm\\OneDrive\\Escritorio\\pythonProject\\SQL---Coderhouse---Proyecto-Final\\csvs\\originales')


In [3]:
books = pd.read_csv('books_data.csv')
ratings = pd.read_csv('Books_rating.csv')

In [ ]:
'''Datos originilales
Books (212404,10)
ratings(30000000,10)'''

In [4]:
#Aca empieza el proceso de normalizacion de las tablas
#Eliminar columnas
books = books.drop(columns=['image','previewLink','infoLink','ratingsCount', 'description'])
ratings = ratings.drop(columns=['review/helpfulness','review/time', 'review/summary', 'review/text'])
#merge de ambas columnas, a fin de eliminar valores y no dejar datos huerfanos
merged_df = pd.merge(books,ratings, on='Title')

In [5]:
# Supongamos que tienes un DataFrame llamado 'books' y quieres verificar la cantidad de celdas con caracteres no deseados en la columna 'Title'
columna_title = merged_df['categories']

# Define la expresión regular para caracteres no deseados
patron = re.compile(r'[^\x00-\x7F]+')

# Cuenta la cantidad de celdas que cumplen la condición (caracteres no deseados en 'Title')
num_celdas_caracteres_no_deseados = columna_title.str.contains(patron).sum()

# Imprime el resultado
print(f"Número de celdas con caracteres no deseados: {num_celdas_caracteres_no_deseados}")




Número de celdas con caracteres no deseados: 660


In [6]:
# Supongamos que tienes un DataFrame llamado 'merged_df' y quieres eliminar las filas con caracteres no deseados en la columna 'authors'

# Define la expresión regular para caracteres no deseados
patron = re.compile(r'[^\x00-\x7F]+')

# Filtra el DataFrame para obtener las filas que cumplen la condición (caracteres no deseados en 'authors')
filas_con_caracteres_no_deseados = merged_df.apply(lambda row: bool(patron.search(str(row['authors']))) and bool(patron.search(str(row['publisher']))) and bool(patron.search(str(row['categories']))), axis=1)


# Elimina las filas que cumplen la condición
merged_df = merged_df[~filas_con_caracteres_no_deseados]






In [7]:
merged_df = merged_df.dropna(subset=['Title']) #elimino filas donde 'Title' es NaN value
merged_df = merged_df.dropna(subset=['User_id', 'profileName'], how='all') #Elimino Nan Values relacionado con usuarios
merged_df = merged_df.dropna(subset=['profileName']) #Elimino Nan Values relacionado con usuarios
merged_df = merged_df.dropna(subset=['publisher', 'publishedDate', 'categories']) #elimino mas Nan values
merged_df = merged_df.dropna(subset=['authors'])#elimino mas Nan values

In [8]:
'''CASO COLUMNA PRICE
Existe una cantidad enorme de Nan Values, pero algunos son recuperables, ya que segun el nro de ID, algunos books tiene precio,
entonces,procedo a completar esos valores que se pueden rellenar'''

# Calcular el valor promedio de Prices para cada book_id
avg_prices = merged_df.groupby('Id')['Price'].mean()

# Rellenar los valores NaN en Prices con el valor promedio para el book_id correspondiente
merged_df['Price'] = merged_df.apply(lambda row: avg_prices[row['Id']] if pd.isna(row['Price']) else row['Price'], axis=1)

#los prices que siguen quedando como Nan, se descartan
merged_df = merged_df.dropna(subset=['Price'])#elimino mas Nan values

'''De esta manera, queda un dataset con 306450 filas por 11 columnas
al cual hay que dividir en 5 tablas'''

'De esta manera, queda un dataset con 306450 filas por 11 columnas\nal cual hay que dividir en 5 tablas'

In [20]:
merged_df.loc[merged_df['profile_name'] == 'Iris Joy Austin']


,title,authors,publisher,published_year,categories,price,profile_name,score,id_book,id_user,id_categories
55503,"Wedding Cakes You Can Make: Designing, Baking,...","Dede Wilson, CCP",Wiley,2005,Cooking,19.77,Iris Joy Austin,5.0,6738,48445,6


In [10]:
# Define el porcentaje de filas a eliminar (por ejemplo, 80%)
porcentaje_eliminar = 0.8

# Calcula el número de filas a eliminar
num_filas_eliminar = int(len(merged_df) * porcentaje_eliminar)

# Realiza la eliminación de forma aleatoria
merged_df = merged_df.iloc[num_filas_eliminar:]  # Elimina las primeras filas según el número calculado

# Verifica el nuevo tamaño del DataFrame
print("Nuevo tamaño del DataFrame:", len(merged_df))


Nuevo tamaño del DataFrame: 61290


In [12]:
merged_df.shape

(61290, 10)

In [14]:
# genero nros de id desde 1
def add_factorize_id_column(df, column_name, new_column_name):
    df[new_column_name] = pd.factorize(df[column_name])[0] + 1
    return df

merged_df = add_factorize_id_column(merged_df, 'Title', 'id_book')
merged_df = add_factorize_id_column(merged_df, 'User_id', 'id_user')
merged_df = add_factorize_id_column(merged_df, 'categories', 'id_categories')

#Elimino mas columnas que no sirven
merged_df = merged_df.drop(columns=['Id', 'User_id'])

In [15]:
#Limpio las columnas de caracteres no deseados
def clean_column(col):
    col = col.str.replace('"', '').str.replace("'", '')
    col = col.str.strip('[]')
    col = col.str.replace('"', '').str.replace("'", '')
    return col

merged_df[['profileName', 'authors', 'categories']] = merged_df[['profileName', 'authors', 'categories']].apply(clean_column)

# Extrae solo el año de la columna 'publishedDate'
merged_df['publishedDate'] = merged_df['publishedDate'].str.extract(r'\b(\d{4})\b')

#cambio el nombre de las columnas
merged_df = merged_df.rename(columns={'Title': 'title', 'publishedDate': 'published_year', 'Price': 'price',
                                      'profileName': 'profile_name', 'review/score': 'score'})

#reinicio el indice
merged_df = merged_df.reset_index(drop=True)

In [16]:
#Separar merged_df en 5 tablas diferentes
def create_table(df, cols, drop_duplicates=None):
    table = df[cols]
    if drop_duplicates:
        table = table.drop_duplicates(subset=drop_duplicates)
    table = table.reset_index(drop=True)
    return table

books = create_table(merged_df, ['id_book', 'title', 'authors', 'publisher', 'published_year', 'id_categories'], 'id_book')
ratings = create_table(merged_df, ['id_user', 'id_book', 'score'])
categories = create_table(merged_df, ['id_categories', 'categories'], 'id_categories')
prices = create_table(merged_df, ['id_book', 'price'], 'id_book')
users = create_table(merged_df, ['id_user', 'profile_name'], 'id_user')


In [17]:
#ultimos arreglos en tabla books:
#quedaron 2 filas con valores null
books = books.dropna()
books['published_year'] = books['published_year'].astype(int) #paso published_year de object a int

In [ ]:
#Dimensiones de las tablas una vez finalizado el proceso
print('books: ', books.shape)
print('ratings: ', ratings.shape)
print('categories: ', categories.shape)
print('prices: ', prices.shape)
print('users: ', users.shape)

In [ ]:
os.chdir('C:\\Users\\juanm\\OneDrive\\Escritorio\\pythonProject\\SQL---Coderhouse---Proyecto-Final\\csvs\\normalizados')

In [ ]:
books.to_csv('books.csv', index=False)
ratings.to_csv('ratings.csv', index=False)
categories.to_csv('categories.csv', index=False)
prices.to_csv('prices.csv', index=False)
users.to_csv('users.csv', index=False)


In [ ]:
merged_df

In [ ]:
columna = 'title'
counts = merged_df[columna].value_counts()
num_celdas_valor_unico = counts[counts == 1].count()
print(num_celdas_valor_unico)


In [ ]:
merged_df.shape

In [ ]:
books.shape